In [1]:

!pip install -U hopsworks --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
import joblib
import os
from PIL import Image

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
import xgboost as xgb

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
import hopsworks

project = hopsworks.login()


fs = project.get_feature_store()


Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/630932
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Retrieve feature groups
data_prediction_fg = fs.get_feature_group(
    name="airquality_data",
    version=1,
)

In [5]:
# Select features for training data

selected_features = data_prediction_fg.select_except(["date", "city", "Humidity","dominant_pollutant"])

In [6]:
selected_features.show(11)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 


,aqi,temperature,humidity,pm25,pm10,o3,no2,co,so2
0,262,21.252898,46.556605,31.466985,85.813875,3.497726,11.967601,4.538414,1.180402
1,104,29.521501,48.540702,16.786433,58.972856,7.591990,17.744047,4.864685,2.236436
2,159,22.482187,46.952460,6.109827,13.752907,9.273984,13.602987,3.829314,0.343409
3,203,28.328122,45.393556,45.593240,33.180568,0.533077,7.509739,1.830997,4.439150
4,26,22.034399,51.514073,44.088754,1.859723,6.900092,7.110304,1.108359,5.799770
5,1,27.957682,56.943305,2.315056,8.561970,0.138129,7.895304,2.566262,0.226488
6,131,24.984302,47.257606,26.526624,56.189580,6.183920,16.038906,3.695787,1.054908
7,21,29.833662,47.619097,7.328156,65.467190,4.093230,3.208962,4.403037,1.274964
8,140,22.750402,47.331605,5.516767,0.426521,8.478827,1.751542,0.292184,0.937389
9,276,22.208445,50.226177,10.989150,87.476451,3.892626,19.279160,4.479257,9.020871


In [7]:
# Load transformation functions from the feature store
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")

In [8]:
# Map features to their corresponding transformation functions
transformation_functions = {}


In [9]:
numerical_features = ["aqi", "temperature", "humidity", "pm25", "pm10", "o3", "no2","co","so2" ]

In [10]:
# For numerical features, use the min_max_scaler transformation
for feature in numerical_features:
    transformation_functions[feature] = min_max_scaler